In [ ]:
%load_ext autoreload
%autoreload 2
import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm
# import mplhep
import matplotlib.pyplot as plt
# mplhep.style.use(mplhep.style.ROOT)
plt.rcParams.update({'font.size': 20,
                        'grid.linestyle': '--',
                        'axes.grid': True,
                        'figure.autolayout': True,
                        'figure.figsize': [14,6]
                        })

import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm
from pathlib import Path

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.BasicWfAna import BasicWfAna
from waffles.data_classes.IPDict import IPDict
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.utils.baseline.baseline import SBaseline
from waffles.np02_utils.AutoMap import generate_ChannelMap, dict_uniqch_to_module, dict_module_to_uniqch, strUch, ordered_channels_membrane, ordered_channels_cathode
from waffles.np02_utils.PlotUtils import np02_gen_grids, plot_grid, plot_detectors, genhist, fithist, runBasicWfAnaNP02
from waffles.np02_utils.PlotUtils import matplotlib_plot_WaveformSetGrid
from waffles.np02_utils.load_utils import ch_read_calib, open_processed
from utils import apply_derivativeWf, apply_moving_averageWf, apply_full_peak_search, apply_peak_search_from_derivative_2, find_peaks_from_derivative_2
from utils import compute_all_peaks, plot_counts, plot_runs, plot_ratio, load_data_search_peaks

In [ ]:
dettype = "membrane"
rates_results = {}


## Only change if necessary
datadir = f"/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/commissioning/"
det = "VD_Cathode_PDS" if dettype == "cathode" else "VD_Membrane_PDS"
endpoint = 106 if dettype == "cathode" else 107
listofch = ordered_channels_cathode if endpoint==106 else ordered_channels_membrane
validchannels = {}
validchannels[106] = ordered_channels_cathode 
validchannels[107] = ordered_channels_membrane
subwaves = 1000 if dettype == "cathode" else 20000

# Way to low... keep scrollng
dletter = dettype.upper()[0] # C or M...
group1 = [ f"{dletter}{detnum}({chnum})" for detnum in range(1, 3) for chnum in range(1,3) ]
group2 = [ f"{dletter}{detnum}({chnum})" for detnum in range(3, 5) for chnum in range(1,3) ]
group3 = [ f"{dletter}{detnum}({chnum})" for detnum in range(5, 7) for chnum in range(1,3) ]
group4 = [ f"{dletter}{detnum}({chnum})" for detnum in range(7, 9) for chnum in range(1,3) ]
groupall = group1+group2+group3+group4

calib_values = ch_read_calib()
list_of_unch = ordered_channels_cathode

In [ ]:
run = 42417
channel = None
nwaveforms=None

In [ ]:
wfset = open_processed(run, dettype, datadir, channel, [107], nwaveforms=nwaveforms, verbose=True, mergefiles=True)

In [ ]:
# wfsetch = ChannelWsGrid.clusterize_waveform_set(wfset)[endpoint]

# timestamp = {}
# for ch, wfs in wfsetch.items():
#     timestamp[ch] = {}
#     for wf in wfs.waveforms:
#         timedaq = wf.daq_window_timestamp
#         ts = wf.timestamp
#         timestamp[ch][timedaq]=timestamp[ch].get(timedaq, [])+[ts]
    
#     timestamp[ch] = {k: sorted(v) for k, v in timestamp[ch].items()}
# # timestamp

In [ ]:
wfsetch = ChannelWsGrid.clusterize_waveform_set(wfset)[endpoint]

timestamp = {}
timestamp_outside = {}
timestamp_inside = {}

points = {}
points_inside = {}
points_outside = {}

for ch, wfs in wfsetch.items():
    if ch not in ordered_channels_membrane:
        continue
    timestamp[ch] = {}
    timestamp_outside[ch] = {}
    timestamp_inside[ch] = {}
    points[ch] = []
    points_inside[ch] = []
    points_outside[ch] = []
    
    for wf in tqdm(wfs.waveforms):
        timedaq = wf.daq_window_timestamp
        ts = wf.timestamp
        timestamp[ch][timedaq]=timestamp[ch].get(timedaq, [])+[ts]
        point = ts - timedaq
        points[ch].append(point)
        if point < -1250 or point > 93750:
            timestamp_outside[ch][timedaq] = timestamp_outside[ch].get(timedaq, [])+[ts] 
            points_outside[ch].append(point)
        # elif point > 93750:
            # timestamp_outside[ch][timedaq] = 
        else: 
            timestamp_inside[ch][timedaq] = timestamp_inside[ch].get(timedaq, [])+[ts]
            points_inside[ch].append(point)
            
    timestamp[ch] = {k: sorted(v) for k, v in timestamp[ch].items()}
    timestamp_inside[ch] = {k: sorted(v) for k, v in timestamp_inside[ch].items()}
    timestamp_outside[ch] = {k: sorted(v) for k, v in timestamp_outside[ch].items()}
    # print(len(timestamp[ch]), len(timestamp_inside[ch]), len(timestamp_outside[ch]))


In [ ]:
diff = {}
diff_in = {}
diff_out = {}

for ch, daq_timestamps in timestamp.items():
    diff[ch] = [ np.diff(v) for v in daq_timestamps.values() ]
    diff[ch] = np.concatenate(diff[ch])

for ch, daq_timestamps in timestamp_inside.items():
    diff_in[ch] = [ np.diff(v) for v in daq_timestamps.values() ]
    diff_in[ch] = np.concatenate(diff_in[ch])

for ch, daq_timestamps in timestamp_outside.items():
    diff_out[ch] = [ np.diff(v) for v in daq_timestamps.values() ]
    diff_out[ch] = np.concatenate(diff_out[ch])

In [ ]:
def plot_diffs(wfset:WaveformSet, diff: dict, diff_in: dict, diff_out:dict, inticks=True):
    endpoint = wfset.waveforms[0].endpoint
    channel = wfset.waveforms[0].channel
    modulename = dict_uniqch_to_module[strUch(endpoint, channel)]

    total_time_all = diff[channel]
    total_time_in = diff_in[channel]
    total_time_out = diff_out[channel]
    
    labelx = '$\Delta$timestamp [ticks]'
    # if not inticks:
        # changing to us
        # total_time_diff = total_time_diff*0.016
        # labelx = r"$\Delta$timestamp [$\mu$s]"
        
    # mintimediff = np.log10(np.min(total_time_all))
    # bins = np.logspace(mintimediff, np.log10(np.max(total_time_all)), 200)
    bins = np.linspace(np.min(total_time_all), np.max(total_time_all), 200)
    # bins = 250
    min_hist = np.min(total_time_all)
    max_hist = np.max(total_time_all)
    plt.hist(total_time_all, bins=bins, histtype="step", label="total wfs")
    plt.hist(total_time_in, bins=bins, histtype="step", label="wfs inside", color='green')
    plt.hist(total_time_out, bins=bins, histtype="step", label="wfs outside", color='red')
    plt.xlabel(labelx)
    # plt.xscale('log')
    # plt.yscale('log')
    plt.title(f"{endpoint}-{ch} {modulename}")
    plt.ylabel('Counts')
    plt.legend(fontsize=15)

In [ ]:
dictparams = dict(
    diff = diff,
    diff_in = diff_in,
    diff_out = diff_out,
)
matplotlib_plot_WaveformSetGrid(wfset, detector=groupall, plot_function=plot_diffs, func_params=dictparams, cols=2, figsize=(14,32))
# plt.savefig(f"hist_delta_timestamps_inout_run{run:06d}")

In [ ]:
def plot_timestamp(wfset:WaveformSet, dict_diff: dict, inticks=True):
    endpoint = wfset.waveforms[0].endpoint
    channel = wfset.waveforms[0].channel
    modulename = dict_uniqch_to_module[strUch(endpoint, channel)]

    min_out = np.min(points_outside[channel])
    max_out = np.max(points_outside[channel])
    bins = 100
    plt.hist(points_inside[channel], range = (min_out, max_out), bins=bins, histtype="step", label="wf inside", color='green')
    plt.hist(points_outside[channel], range = (min_out, max_out), bins=bins, histtype="step", label="wf outside", color='red')
    
    plt.xlabel("wf timestamp - daq timestamp")
    plt.ylabel("Counts")
    plt.title(f"{endpoint}-{ch} {modulename}")
    plt.legend(fontsize=15)


In [ ]:
dictparams = dict(
    dict_diff = diff
)
matplotlib_plot_WaveformSetGrid(wfset, detector=group3+group4, plot_function=plot_timestamp, func_params=dictparams, cols=2, figsize=(14,16))
# plt.savefig(f"hist_timestamps_inout_run{run:06d}")

In [ ]:
dictparams["inticks"] = False
matplotlib_plot_WaveformSetGrid(wfset, detector=group3+group4, plot_function=plot_diffs, func_params=dictparams, cols=2, figsize=(14,16))
# plt.savefig(f"hist_delta_timestamps_microseconds_run{run:06d}")

In [ ]:
def plot_ratios(wfset:WaveformSet, diff: dict, inticks=True):
    endpoint = wfset.waveforms[0].endpoint
    channel = wfset.waveforms[0].channel
    modulename = dict_uniqch_to_module[strUch(endpoint, channel)]

    total_time_all = diff[channel]
    
    labelx = '$\Delta$timestamp [ticks]'
    # if not inticks:
        # changing to us
        # total_time_diff = total_time_diff*0.016
        # labelx = r"$\Delta$timestamp [$\mu$s]"
        
    mintimediff = np.log10(np.min(total_time_all))
    bins = np.logspace(mintimediff, np.log10(np.max(total_time_all)), 200)
    # bins = np.linspace(np.min(total_time_all), np.max(total_time_all), 200)
    counts, bins = np.histogram(total_time_all, bins=bins)
    # counts_sum = np.cumsum(counts)[::-1]
    counts_cum = np.cumsum( counts[::-1] )[::-1]
    tot = np.sum(counts)
    plt.hist(bins[:-1], bins, weights=100*counts_cum/tot,  histtype="step")
    #plt.hist(total_time_all, bins=bins, histtype="step", label="total wfs")
    plt.xlabel(labelx)
    plt.xscale('log')
    plt.yscale('log')
    plt.title(f"{endpoint}-{ch} {modulename}")
    plt.ylabel('Fraction [%]')

In [ ]:
matplotlib_plot_WaveformSetGrid(wfset, detector=groupall, plot_function=plot_ratios, func_params={"diff":diff}, cols=2, figsize=(14,32))
plt.savefig(f"hist_delta_timestamps_fract_run{run:06d}_loglog")